# XYG3 梯度

在这一节中，我们会回顾以 GGA 为参考态的梯度性质计算，并最终得到 XYG3 梯度。

In [1]:
import numpy as np
from pyscf import scf, gto, lib, grad, hessian, dft, mp
import pyscf.hessian.rks
import pyscf.grad.rks
from functools import partial

from utilities import val_from_fchk, NumericDiff
from hessian import HFHelper, GGAHelper, NCGGAEngine

np.einsum = partial(np.einsum, optimize=["greedy", 1024 ** 3 * 2 / 8])
np.einsum_path = partial(np.einsum_path, optimize=["greedy", 1024 ** 3 * 2 / 8])
np.set_printoptions(6, linewidth=150, suppress=True)

In [2]:
mol = gto.Mole()
mol.atom = """
O  0.0  0.0  0.0
O  0.0  0.0  1.5
H  1.5  0.0  0.0
H  0.0  0.7  1.5
"""
mol.basis = "6-31G"
mol.verbose = 0
mol.build()

def mol_to_grids(mol):
    grids = dft.gen_grid.Grids(mol)
    grids.atom_grid = (75, 302)
    grids.becke_scheme = dft.gen_grid.stratmann
    grids.prune = None
    grids.build()
    return grids
grids = mol_to_grids(mol)

nmo = nao = mol.nao
natm = mol.natm
nocc = mol.nelec[0]
nvir = nmo - nocc
so = slice(0, nocc)
sv = slice(nocc, nmo)
sa = slice(0, nmo)

下面将生成自洽场助手 `scfh`、非自恰泛函 `nch` 与 GGA 非自恰助手 `ncgga`。

In [3]:
scfh = GGAHelper(mol, "b3lypg", grids)
nch = GGAHelper(mol, "0.8033*HF - 0.0140*LDA + 0.2107*B88, 0.6789*LYP", mol_to_grids(mol), init_scf=False)
ncgga = NCGGAEngine(scfh, nch)

## 参考值

XYG3 梯度可以分为其非自恰的 GGA 能量与核排斥能量 $E_\mathrm{elec}^\mathrm{HF} + E_\mathrm{elec}^\mathrm{GGA} + E_\mathrm{nuc}$ 的梯度贡献与 PT2 能量 $E_\mathrm{elec}^\mathrm{MP2}$ 的梯度贡献。这里的记号暂时就不按照正式文献 (Su, JCC 2013) 的走法了。

我们以前的文档已经详细地描述如何求取 $\frac{\partial}{\partial A_t} (E_\mathrm{elec}^\mathrm{HF} + E_\mathrm{elec}^\mathrm{GGA} + E_\mathrm{nuc})$ 了。下面的代码就是求取了 XYG3 泛函中，除了 PT2 的梯度贡献：

In [4]:
ncgga.E_1

array([[-0.13754 ,  0.016505, -0.018595],
       [ 0.009104,  0.722476,  0.044859],
       [ 0.12171 ,  0.003139,  0.01583 ],
       [ 0.006726, -0.742119, -0.042093]])

我们下面的任务便是求取 PT2 部分的贡献。

用于参考的 [XYG3 梯度](include/mp2_grad/xyg3_grad.gjf) 通过谷永浩在 Gaussian 09 修改的程序所得到：

In [5]:
grad_xyg3_ref = val_from_fchk("Cartesian Gradient", "include/mp2_grad/xyg3_grad.fchk").reshape((natm, 3))
grad_xyg3_ref

array([[-0.102274,  0.014222,  0.023366],
       [ 0.008586,  0.740529, -0.001475],
       [ 0.087769,  0.002763,  0.014506],
       [ 0.00592 , -0.757514, -0.036397]])

如果我们已经考虑了 XYG3 下 PT2 贡献的系数 $c_\mathrm{c} = 0.3211$，那么如果 B3LYP 作为参考态所作的 MP2 能量的梯度应当是下述值：

In [6]:
grad_B3LYP_MP2_ref = (grad_xyg3_ref - ncgga.E_1) / 0.3211
grad_B3LYP_MP2_ref

array([[ 0.109827, -0.007112,  0.130679],
       [-0.001616,  0.056224, -0.144299],
       [-0.105702, -0.00117 , -0.004122],
       [-0.00251 , -0.047943,  0.01774 ]])

后文的主要目的就是重复该梯度值。

## XYG3 梯度实现

### 分项生成

我们首先生成必要的矩阵与张量。这里与以 HF 为参考态的 MP2 并无很大区别；只是我们不再生成完整的双粒子密度，也不使用 Hamiltonian Core Skeleton 梯度；取而代之的是使用 Fock 矩阵的 Skeleton 梯度 `F_1_mo` $F_{pq}^{A_t}$：

In [7]:
%%capture

e, eo, ev = scfh.e, scfh.eo, scfh.ev
C, Co, Cv = scfh.C, scfh.Co, scfh.Cv
D = scfh.D
eri0_mo = scfh.eri0_mo
eri1_ao = scfh.eri1_ao

S_1_mo = scfh.S_1_mo
F_1_mo = scfh.F_1_mo
Ax0_Core = scfh.Ax0_Core

D_iajb = lib.direct_sum("i - a + j - b", scfh.eo, scfh.ev, scfh.eo, scfh.ev)
t_iajb = eri0_mo[so, sv, so, sv] / D_iajb
T_iajb = 2 * t_iajb - t_iajb.swapaxes(1, 3)

生成弛豫密度 `D_r` $D_{pq}^\mathrm{MP2}$、加权密度 `D_W` $W_{pq}^\mathrm{MP2}$ 与不可拆分双粒子密度 `D_pdm2_NS` $\Gamma_{\mu \nu \kappa \lambda}^\mathrm{MP2, NS}$ 的过程与 HF 参考态下也完全一致：

In [9]:
D_r = np.zeros((nmo, nmo))
D_r[so, so] += - 2 * np.einsum("iakb, jakb -> ij", T_iajb, t_iajb)
D_r[sv, sv] += 2 * np.einsum("iajc, ibjc -> ab", T_iajb, t_iajb)

L = np.zeros((nvir, nocc))
L += Ax0_Core(sv, so, sa, sa)(D_r)
L -= 4 * np.einsum("jakb, ijbk -> ai", T_iajb, eri0_mo[so, so, sv, so])
L += 4 * np.einsum("ibjc, abjc -> ai", T_iajb, eri0_mo[sv, sv, so, sv])

D_r[sv, so] = scf.cphf.solve(Ax0_Core(sv, so, sv, so), e, scfh.mo_occ, L, max_cycle=100, tol=1e-13)[0]

In [10]:
# W[I] - Correct with s1-im1 term in PySCF
D_WI = np.zeros((nmo, nmo))
D_WI[so, so] = - 2 * np.einsum("iakb, jakb -> ij", T_iajb, eri0_mo[so, sv, so, sv])
D_WI[sv, sv] = - 2 * np.einsum("iajc, ibjc -> ab", T_iajb, eri0_mo[so, sv, so, sv])
D_WI[sv, so] = - 4 * np.einsum("jakb, ijbk -> ai", T_iajb, eri0_mo[so, so, sv, so])

# W[II] - Correct with s1-zeta term in PySCF
# Note that zeta in PySCF includes HF energy weighted density rdm1e
# The need of scaler 1 in D_WII[sv, so] is that Aikens use doubled P
D_WII = np.zeros((nmo, nmo))
D_WII[so, so] = - 0.5 * D_r[so, so] * lib.direct_sum("i + j -> ij", eo, eo)
D_WII[sv, sv] = - 0.5 * D_r[sv, sv] * lib.direct_sum("a + b -> ab", ev, ev)
D_WII[sv, so] = - D_r[sv, so] * eo

# W[III] - Correct with s1-vhf_s1occ term in PySCF
D_WIII = np.zeros((nmo, nmo))
D_WIII[so, so] = - 0.5 * Ax0_Core(so, so, sa, sa)(D_r)

# Summation
D_W = D_WI + D_WII + D_WIII

In [11]:
# Non-seperatable - Correct with `de` generated in PySCF code part --2e AO integrals dot 2pdm--
D_pdm2_NS = 2 * np.einsum("iajb, ui, va, kj, lb -> uvkl", T_iajb, Co, Cv, Co, Cv)

### 分项求和

我们求以 B3LYP 为参考态的 MP2 梯度时，不能再使用 HF 参考态的公式

$$
E_\mathrm{elec}^{\mathrm{MP2}, A_t} = D_{pq}^\mathrm{MP2} h_{pq}^{A_t} + W_{pq}^\mathrm{MP2} S_{pq}^{A_t} + \Gamma_{\mu \nu \kappa \lambda}^\mathrm{MP2} (\mu \nu | \kappa \lambda)^{A_t}
$$

这是因为，对于 HF 参考态，

$$
\Gamma_{\mu \nu \kappa \lambda}^\mathrm{MP2, NS} (\mu \nu | \kappa \lambda)^{A_t} = \big( D_{pq}^\mathrm{MP2} C_{\mu p} C_{\nu q} D_{\kappa \lambda} - \frac{1}{2} D_{pq}^\mathrm{MP2} C_{\mu p} C_{\kappa q} D_{\nu \lambda} \big) (\mu \nu | \kappa \lambda)^{A_t} \quad \text{(HF reference)}
$$

上述一项是从 Fock 矩阵 Skeleton 导数所派生得来。我们应当会注意到，对于 HF 参考态而言，

$$
D_{pq}^\mathrm{MP2} F_{pq}^{A_t} = D_{pq}^\mathrm{MP2} h_{pq}^{A_t} + \Gamma_{\mu \nu \kappa \lambda}^\mathrm{MP2, NS} (\mu \nu | \kappa \lambda)^{A_t} \quad \text{(HF, GGA reference)}
$$

这个公式对以 B3LYP 为参考态的 MP2 也一样适用。因此，我们使用 $D_{pq}^\mathrm{MP2} F_{pq}^{A_t}$ 来替代 $\Gamma_{\mu \nu \kappa \lambda}^\mathrm{MP2, NS}$ 与 $h_{pq}^{A_t}$ 两项所产生的贡献：

$$
E_\mathrm{elec}^{\mathrm{MP2}, A_t} = D_{pq}^\mathrm{MP2} F_{pq}^{A_t} + W_{pq}^\mathrm{MP2} S_{pq}^{A_t} + \Gamma_{\mu \nu \kappa \lambda}^\mathrm{MP2, NS} (\mu \nu | \kappa \lambda)^{A_t} \quad \text{(HF, GGA reference)}
$$

In [12]:
grad_B3LYP_MP2 = (
    + (D_r * F_1_mo).sum(axis=(-1, -2))
    + (D_W * S_1_mo).sum(axis=(-1, -2))
    + (D_pdm2_NS * eri1_ao).sum(axis=(-1, -2, -3, -4))
)
grad_B3LYP_MP2

array([[ 0.109826, -0.007108,  0.130685],
       [-0.001614,  0.056212, -0.144301],
       [-0.105701, -0.00117 , -0.004126],
       [-0.002508, -0.047933,  0.017739]])

我们可以验证上述的计算结果确实是参考结果所给出的 B3LYP 参考态下的 MP2 相关能梯度，但需要稍微把判断标准放低一些。

In [13]:
np.allclose(grad_B3LYP_MP2, grad_B3LYP_MP2_ref, atol=1e-5, rtol=1e-4)

True

由此，XYG3 梯度可以由下述代码给出：

In [14]:
ncgga.E_1 + 0.3211 * grad_B3LYP_MP2_ref

array([[-0.102274,  0.014222,  0.023366],
       [ 0.008586,  0.740529, -0.001475],
       [ 0.087769,  0.002763,  0.014506],
       [ 0.00592 , -0.757514, -0.036397]])

In [15]:
np.allclose(
    ncgga.E_1 + 0.3211 * grad_B3LYP_MP2,
    grad_xyg3_ref,
    atol=1e-6, rtol=1e-4
)

True

最后，我们指出，上述代码除开比较大内存靠小意外，也这不是生成 XYG3 梯度的最高效的方案，因为上述过程执行了两次 Z-Vector 和两次与 $(\mu \nu | \kappa \lambda)^{A_t}$ 有关的计算。因此，在真正的程序实践中 (譬如 Su, JCC, 2013)，应当要与非自恰 GGA 梯度的计算过程整合起来。